In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

ImportError: Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
df = pd.read_csv()

In [ ]:
# Requirements: tensorflow (>=2.6), scikit-learn, numpy, matplotlib


# -------------------------
# 1) Load your data
# -------------------------
# Replace this with your actual loading code
# X_raw should be shape (n_samples, 178)
# y should be shape (n_samples,) with values 0 or 1
# Optionally: subject_ids shape (n_samples,) if available for subject-wise splitting
# Example placeholder:
# X_raw = np.load("X.npy")
# y = np.load("y.npy")
# subject_ids = np.load("subject_ids.npy")  # optional
#
# For illustration, here's random data — replace with real dataset:
# np.random.seed(0)
# X_raw = np.random.randn(2000, 178)
# y = np.random.choice([0,1], size=(2000,), p=[0.8,0.2])

# -------------------------
# 2) Preprocessing helpers
# -------------------------
def standardize(X, axis=1):
    # z-score per-sample along time axis
    mean = X.mean(axis=axis, keepdims=True)
    std = X.std(axis=axis, keepdims=True) + 1e-8
    return (X - mean) / std

def augment_noise(X, noise_level=0.01):
    return X + noise_level * np.random.randn(*X.shape)

# -------------------------
# 3) Prepare inputs
# -------------------------
# Example: standardize and reshape for Conv1D/LSTM: (n_samples, timesteps, channels)
X = standardize(X_raw, axis=1)
X = X[..., np.newaxis]   # shape -> (n_samples, 178, 1)

# train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, stratify=y, random_state=42
)

# Optionally set class weights for imbalance
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = {int(c): w for c, w in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# -------------------------
# 4) Model architecture
# -------------------------
def make_cnn_lstm_model(
    timesteps=178,
    channels=1,
    conv_filters=[64, 128],
    conv_kernel_sizes=[7,5],
    pool_sizes=[2,2],
    lstm_units=64,
    dense_units=64,
    dropout_rate=0.4,
    l2_reg=1e-4
):
    inp = layers.Input(shape=(timesteps, channels))
    x = inp

    # Conv block 1
    x = layers.Conv1D(filters=conv_filters[0], kernel_size=conv_kernel_sizes[0], padding='same',
                      kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(pool_size=pool_sizes[0])(x)
    x = layers.Dropout(0.15)(x)

    # Conv block 2
    x = layers.Conv1D(filters=conv_filters[1], kernel_size=conv_kernel_sizes[1], padding='same',
                      kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(pool_size=pool_sizes[1])(x)
    x = layers.Dropout(0.15)(x)

    # Optionally more conv blocks here...

    # Bidirectional LSTM
    x = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=False, recurrent_dropout=0.15))(x)

    x = layers.Dense(dense_units, kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(dropout_rate)(x)

    out = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=inp, outputs=out)
    return model

model = make_cnn_lstm_model()
model.summary()

# -------------------------
# 5) Compile
# -------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

# -------------------------
# 6) Callbacks
# -------------------------
cb_early = callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1)
cb_reduce = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
cb_checkpoint = callbacks.ModelCheckpoint('best_cnn_lstm.h5', monitor='val_loss', save_best_only=True, verbose=1)

# -------------------------
# 7) Training, with augmentation (optional)
# -------------------------
# Simple augmentation generator
def data_generator(X, y, batch_size=64, augment=False):
    n = len(X)
    idx = np.arange(n)
    while True:
        np.random.shuffle(idx)
        for i in range(0, n, batch_size):
            batch_idx = idx[i:i+batch_size]
            Xb = X[batch_idx].copy()
            yb = y[batch_idx]
            if augment:
                # example augmentations: small noise, slight scaling
                Xb = augment_noise(Xb, noise_level=0.01)
                # other augmentations: time shift, random crop, scale, etc.
            yield Xb, yb

batch_size = 64
epochs = 80

# Use generator for training to enable augmentation
train_gen = data_generator(X_train, y_train, batch_size=batch_size, augment=True)
steps_per_epoch = int(np.ceil(len(X_train) / batch_size))

history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_test, y_test),
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[cb_early, cb_reduce, cb_checkpoint],
    verbose=2
)

# -------------------------
# 8) Evaluate on test set
# -------------------------
y_pred_prob = model.predict(X_test, batch_size=128).ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
cm = confusion_matrix(y_test, y_pred)

print(f"Test accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}  Recall: {rec:.4f}  F1: {f1:.4f}")
print("Confusion Matrix:\n", cm)
